In [3]:
#Librairie pour l'analyse du texte

import pandas as pd
from pandas import *

import numpy as np
import time

import sklearn
from sklearn import *
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn import (manifold, datasets, decomposition, ensemble, discriminant_analysis, random_projection)
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
# from sklearn import metrics
# from sklearn import linear_model
from sklearn.decomposition import TruncatedSVD

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import *

import threading

In [18]:
import langdetect

ImportError: No module named langdetect

In [4]:
# ---------------------------------- 1er choix

# data = pd.read_csv('\Users\k_lemo\Documents\Stage\Data_EPO\data2_0.csv')

# ---------------------------------- 2ième choix

data = pd.read_csv('\Users\k_lemo\Documents\Stage\Data_EPO\BIG\data1_1.csv', iterator=True, chunksize=1000)
data = concat(data, ignore_index=True)
data = data.drop(['cpc_2'], axis=1)

In [5]:
data

,doc_id,cpc_1,full_text
0,294490073,G11B,Darstellung des Brockens und seiner Umgebu...
1,296906943,G06M,Verwendung einer Schablone mit mehreren Au...
2,294827982,A63H,Das Spiel besteht aus Holzteilen in de...
3,328236491,D08303000O,CLAIMThe ornamental design for a handlebar gri...
4,325709882,074015000X,It H J HABER METHOD OF TREATING CHARCOA...
5,376604586,C09B,The red pigment preparation method of Fructus ...
6,376607600,A24D,A cigarette have rough cylindrical tobacco co...
7,333873740,H02H,nichtig nichtigEinklemmsicherungssystem nach A...
8,406442150,A61C,Gerätesatz umfassend eine Anzahl von unterschi...
9,293893551,E04H,Kompakte komplett transportabele fabrikfer...


In [6]:
# Suppression des STOP WORDS :
# On télécharge l'ensemble des stopwords --> http://www.nltk.org/nltk_data/
# On rassemble "à la main" l'ensemble des stopwords de tous les languages téléchargés (C:\Users\k_lemo\Documents\Stage\stopwords\All.txt)

stopwords = pd.read_csv("\Users\k_lemo\Documents\Stage\stopwords\All.txt", header=None, sep=r"\s+") 
stopWords = [word for word in set(stopwords[0])]
stopwords


# Piste d'optimisation : les stopwords avec des caractères spéciaux ne peuvent pas être détectés lorsqu'il sont sous forme de dataframe
# --> Appliquer le filtre sur le fichier en dur mais cela ralentira considérablement le process

,0
0,og
1,i
2,jeg
3,det
4,at
5,en
6,den
7,til
8,er
9,som


In [12]:
#On Vectorise le champs full_text à l'aide de l'algorithme TF-IDF

start_time = time.time()
tfIdf = TfidfVectorizer(analyzer = "word", stop_words=stopWords,max_features = 25).fit_transform(data['full_text'])
print("--- %s seconds ---" % (time.time() - start_time))

tfIdf

--- 154.832000017 seconds ---


<40000x25 sparse matrix of type '<type 'numpy.float64'>'
	with 703843 stored elements in Compressed Sparse Row format>

In [ ]:
# Classification Random Forest


# Définition de la pipeline
rf_clf = Pipeline([('vect', TfidfVectorizer(analyzer = "word",stop_words=stopWords,max_features = 1000)),
                   ('clf', RandomForestClassifier(n_estimators = 150,n_jobs=-1))])

# Chargement des données d'apprentissage et des données à analyser
data_80 = data.sample(frac=0.8)
data_20 = data.loc[~data.index.isin(data_80.index)]

# Apprentissage
tps1 = time.clock()
rf_clf = rf_clf.fit(data_80["full_text"],data_80["cpc_1"])
tps2 = time.clock() 
print "Training done in ",tps2 - tps1," seconds."
print""

# Prédictions
tps1 = time.clock()
result = rf_clf.predict(data_20["full_text"])
tps2 = time.clock() 
print "Predictions done in ",tps2 - tps1," seconds."
print""

# On sauvegarde le resultat
# output = pd.DataFrame( data={"id":data_20["doc_id"], "cpc_1":result} )
# output.to_csv( "\Users\k_lemo\Documents\Stage\Data_EPO\resultat_RF.csv", index=False, quoting=3)

# On calcule le taux de bonnes prédictions
print 'Match rate: ',np.mean(result == data_20["cpc_1"])*100,"%"
print''
print 'Confusion Matrix:'
metrics.confusion_matrix(data_20["cpc_1"],result)

In [15]:
# Projection 2D via python threading

class ThreadClass(threading.Thread):
  def run(self):
    now = datetime.datetime.now()
    print "%s says Hello World at time: %s" % 
    (self.getName(), now)

for i in range(2):
  t = ThreadClass()
  t.start()

In [14]:
# Actuellement constituée de vecteurs de dimension 100, la représentation de TfIdf est impossible sans une projection en espace bidimensionnel 

start_time = time.time()
tsne = TSNE(n_components=2, init='pca',random_state=0)
X1 = tsne.fit_transform(tfIdf2=tfIdf.toarray())
print("--- %s seconds ---" % (time.time() - start_time))

for i in range(len(data.index)):
    #plt.plot(clusters[0][i][0], clusters[0][i][1],'.',color=plt.cm.Set1(clusters[1][i] / float(centers)))
    plt.plot(X1[:,0],X1[:,1],'.',color=plt.cm.Set1(data["CPC1_corresp"][i] / float(len(data["CPC1_corresp"]))))

#X1
#X1[:,0]

MemoryError: 

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(len(data.index)):
    #plt.plot(clusters[0][i][0], clusters[0][i][1],'.',color=plt.cm.Set1(clusters[1][i] / float(centers)))
    plt.plot(X1[i][0],X1[i][1],'.',color=plt.cm.Set1(data["CPC1_corresp"][i] / float(len(set(data["CPC1_corresp"])))))

In [11]:
X_reduced = TruncatedSVD(n_components=50, random_state=0).fit_transform(tfIdf)
X_embedded = TSNE(n_components=2, perplexity=40, verbose=2).fit_transform(X_reduced)

[t-SNE] Computing pairwise distances...


MemoryError: 

In [ ]:
plt.figure(figsize=(10, 10))
plt.setp(plt.axes(frameon=False), xticks=(), yticks=())
plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9, wspace=0.0, hspace=0.0)
for i in range(len(data.index)):
    plt.plot(X_embedded[i][0],X_embedded[i][1],'.',color=plt.cm.Set1(data["CPC1_corresp"][i] / float(len(set(data["CPC1_corresp"])))))